In [145]:

import numpy as np
import pandas as pd
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, LineString, box
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple, Optional
import networkx as nx
from dataclasses import dataclass
import rtree
import logging
from tqdm import tqdm
from math import pi
from typing import List, Dict, Optional
import seaborn

import ast
import os
import folium
from datetime import datetime
from folium import plugins
from branca.colormap import LinearColormap

In [146]:
# Adjusted Constants for improved confidence
SIGMA_Z = 15.0  # Increased sigma_z for more tolerance in emission
MAX_DISTANCE = 50.0  # Increased max_distance for broader candidate search
TURN_ANGLE_THRESHOLD = pi / 4  # 45 degrees threshold for transition penalty
MIN_TRANSITION_PROB = 1e-5  # Non-zero transition probability for flexibility

def process_trajectory(polyline_str: str) -> List[tuple]:
    """Process trajectory string into coordinates with more lenient validation"""
    try:
        if not isinstance(polyline_str, str):
            return None
        coords = ast.literal_eval(polyline_str)
        if not coords:
            return None
        
        # More lenient validation - allow trajectories with at least 2 points
        valid_coords = []
        for coord in coords:
            if len(coord) == 2:
                # More forgiving coordinate validation
                x, y = coord
                if isinstance(x, (int, float)) and isinstance(y, (int, float)):
                    # Wider coordinate bounds
                    if -180 <= x <= 180 and -90 <= y <= 90:
                        valid_coords.append(coord)
        
        return valid_coords if len(valid_coords) >= 2 else None
    except Exception as e:
        logging.warning(f"Error processing trajectory: {str(e)}")
        return None






class EnhancedViterbiMatcher:
    def __init__(self, graph, edges_gdf, config=None):
        """Initialize matcher with improved configuration"""
        self.graph = graph
        self.edges_gdf = edges_gdf.copy()
        
        if isinstance(self.edges_gdf.index, pd.MultiIndex):
            self.edges_gdf = self.edges_gdf.reset_index(drop=True)
        self.edges_gdf.index = range(len(self.edges_gdf))
        
        # Enhanced default configuration
        default_config = {
            'max_candidates': 20,          # Increased from 10
            'max_distance': 100.0,         # Increased from 50.0
            'sigma_z': 50.0,              # Adjusted for better GPS noise handling
            'beta': 2.0,                  # Increased for better transition scoring
            'min_prob_norm': 1e-7,        # Lowered for more flexibility
            'max_speed': 50.0,            # Maximum expected speed (m/s)
            'min_speed': 0.1,             # Minimum expected speed (m/s)
            'angle_tolerance': np.pi/2,    # 90 degrees angle tolerance
            'max_angle_penalty': 0.5,      # Maximum penalty for sharp turns
            'distance_decay': 0.85,        # Distance decay factor
            'sequential_matching': True    # Enable sequential matching for long trajectories
        }
        
        if config:
            default_config.update(config)
        self.config = default_config
        
        self._init_spatial_index()
        self.edge_to_nodes = self._build_edge_to_nodes()
        self.node_to_edges = self._build_node_to_edges()
        
        self.logger = logging.getLogger(__name__)
    
    def _init_spatial_index(self):
        """Initialize R-tree spatial index with improved error handling"""
        try:
            self.spatial_index = rtree.index.Index()
            for idx, edge in self.edges_gdf.iterrows():
                if edge.geometry is not None and not edge.geometry.is_empty:
                    self.spatial_index.insert(idx, edge.geometry.bounds)
        except Exception as e:
            self.logger.error(f"Error initializing spatial index: {str(e)}")
            raise

    def _build_edge_to_nodes(self) -> Dict[int, set]:
        """Build mapping from edge IDs to their endpoint nodes with validation"""
        edge_to_nodes = {}
        for idx, edge in self.edges_gdf.iterrows():
            if edge.geometry is not None and not edge.geometry.is_empty:
                coords = list(edge.geometry.coords)
                if len(coords) >= 2:  # Ensure valid linestring
                    edge_to_nodes[idx] = {
                        self._get_node_id(coords[0]),
                        self._get_node_id(coords[-1])
                    }
        return edge_to_nodes

    def _build_node_to_edges(self) -> Dict[tuple, set]:
        """Build mapping from nodes to connected edge IDs"""
        node_to_edges = {}
        for edge_id, nodes in self.edge_to_nodes.items():
            for node in nodes:
                if node not in node_to_edges:
                    node_to_edges[node] = set()
                node_to_edges[node].add(edge_id)
        return node_to_edges

    def _get_node_id(self, coord: tuple) -> tuple:
        """Convert coordinate to node ID with improved precision"""
        return tuple(round(x, 6) for x in coord)

    def _find_candidates(self, point: Point) -> List[dict]:
        """Enhanced candidate finding with adaptive search radius"""
        candidates = []
        initial_distance = self.config['max_distance']
        max_attempts = 3
        current_distance = initial_distance
        
        for attempt in range(max_attempts):
            bounds = (
                point.x - current_distance,
                point.y - current_distance,
                point.x + current_distance,
                point.y + current_distance
            )
            
            for idx in self.spatial_index.intersection(bounds):
                edge = self.edges_gdf.loc[idx]
                if edge.geometry is not None:
                    dist = point.distance(edge.geometry)
                    if dist <= current_distance:
                        proj_point = edge.geometry.interpolate(
                            edge.geometry.project(point)
                        )
                        candidates.append({
                            'edge_id': idx,
                            'distance': dist,
                            'proj_point': proj_point,
                            'edge': edge
                        })
            
            if candidates:
                break
                
            current_distance *= 1.5  # Increase search radius for next attempt
        
        # Sort by distance and apply adaptive limit
        candidates.sort(key=lambda x: x['distance'])
        return candidates[:self.config['max_candidates']]

    def _calculate_emission_prob(self, point: Point, candidate: dict) -> float:
        """Enhanced emission probability calculation with improved scaling"""
        distance = candidate['distance']
        sigma_z = self.config['sigma_z']
        
        # Distance-based probability with decay
        distance_factor = np.exp(-distance * self.config['distance_decay'])
        
        # Gaussian probability
        gaussian_prob = np.exp(-0.5 * (distance / sigma_z) ** 2)
        
        # Combined probability
        prob = gaussian_prob * distance_factor
        
        return max(prob, self.config['min_prob_norm'])

    def _calculate_transition_prob(self, prev_edge: int, curr_edge: int,
                                 prev_point: Point, curr_point: Point) -> float:
        """Enhanced transition probability calculation with improved angle handling"""
        prev_nodes = self.edge_to_nodes[prev_edge]
        curr_nodes = self.edge_to_nodes[curr_edge]
        
        # Check connectivity with more flexibility
        connected = bool(prev_nodes.intersection(curr_nodes))
        connectivity_score = 1.0 if connected else 0.3
        
        # Calculate angle similarity
        dir1 = np.array(prev_point.coords[-1]) - np.array(prev_point.coords[0])
        dir2 = np.array(curr_point.coords[-1]) - np.array(curr_point.coords[0])
        
        norm1 = np.linalg.norm(dir1)
        norm2 = np.linalg.norm(dir2)
        
        if norm1 == 0 or norm2 == 0:
            angle_score = 1.0
        else:
            cos_angle = np.dot(dir1, dir2) / (norm1 * norm2)
            angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
            
            # Smoother angle penalty
            angle_score = 1.0 - (angle / self.config['angle_tolerance']) * self.config['max_angle_penalty']
            angle_score = max(angle_score, 1.0 - self.config['max_angle_penalty'])
        
        # Combined probability
        prob = connectivity_score * angle_score
        
        return max(prob, self.config['min_prob_norm'])

    def _viterbi_matching(self, points: List[Point], candidates_by_point: List[List[dict]]) -> List[Dict]:
        """Improved Viterbi algorithm with better numerical stability"""
        n_points = len(points)
        states = [{} for _ in range(n_points)]
        
        # Initialize first state with log probabilities
        for candidate in candidates_by_point[0]:
            edge_id = candidate['edge_id']
            log_emission = np.log(self._calculate_emission_prob(points[0], candidate))
            states[0][edge_id] = {
                'log_prob': log_emission,
                'prev': None,
                'emission': log_emission,
                'transition': 0.0
            }
        
        # Forward pass with log probabilities
        for t in range(1, n_points):
            for candidate in candidates_by_point[t]:
                curr_edge = candidate['edge_id']
                log_emission = np.log(self._calculate_emission_prob(points[t], candidate))
                
                best_log_prob = float('-inf')
                best_prev = None
                best_transition = None
                
                for prev_edge, prev_state in states[t-1].items():
                    trans_prob = self._calculate_transition_prob(
                        prev_edge, curr_edge, points[t-1], points[t]
                    )
                    log_transition = np.log(trans_prob)
                    
                    log_prob = prev_state['log_prob'] + log_transition + log_emission
                    
                    if log_prob > best_log_prob:
                        best_log_prob = log_prob
                        best_prev = prev_edge
                        best_transition = log_transition
                
                if best_prev is not None:
                    states[t][curr_edge] = {
                        'log_prob': best_log_prob,
                        'prev': best_prev,
                        'emission': log_emission,
                        'transition': best_transition
                    }
        
        # Convert log probabilities to normalized confidence scores
        if states[-1]:
            log_probs = np.array([state['log_prob'] for state in states[-1].values()])
            max_log_prob = np.max(log_probs)
            normalized_probs = np.exp(log_probs - max_log_prob)
            normalized_probs /= np.sum(normalized_probs)
            
            for edge_id, norm_prob in zip(states[-1].keys(), normalized_probs):
                states[-1][edge_id]['confidence'] = norm_prob
        
        return states

    def _backtrack(self, states: List[Dict]) -> List[int]:
        """Backtrack to find the best path with improved handling of edge cases"""
        if not states or not states[-1]:
            return []
        
        path = []
        current_edge = max(states[-1].items(), key=lambda x: x[1]['log_prob'])[0]
        
        for t in range(len(states) - 1, -1, -1):
            path.append(current_edge)
            if t > 0 and states[t][current_edge]['prev'] is not None:
                current_edge = states[t][current_edge]['prev']
        
        return list(reversed(path))

    def _sequential_matching(self, points: List[Point]) -> Dict:
        """Match long trajectories in sequential segments with overlap"""
        segment_size = 30
        overlap = 10
        all_edges = []
        segment_confidences = []
        
        for i in range(0, len(points), segment_size - overlap):
            segment = points[i:i + segment_size]
            if len(segment) < 2:
                continue
                
            candidates = [self._find_candidates(p) for p in segment]
            if not all(candidates):
                continue
                
            states = self._viterbi_matching(segment, candidates)
            path = self._backtrack(states)
            
            if path:
                if states[-1] and path[-1] in states[-1]:
                    segment_confidences.append(states[-1][path[-1]].get('confidence', 0.0))
                    
                # Remove overlap with previous segment
                if all_edges and overlap > 0:
                    all_edges = all_edges[:-overlap]
                all_edges.extend(path)
        
        if not all_edges:
            return {'success': False, 'edges': [], 'confidence': 0.0}
        
        # Calculate overall confidence as average of segment confidences
        overall_confidence = np.mean(segment_confidences) if segment_confidences else 0.0
            
        return {
            'success': True,
            'edges': all_edges,
            'confidence': overall_confidence
        }

    def match_trajectory(self, points: List[Tuple[float, float]]) -> Dict:
        """Match trajectory with improved error handling and validation"""
        try:
            if len(points) < 2:
                return {'success': False, 'edges': [], 'confidence': 0.0}

            point_objects = [Point(p) for p in points]
            
            # Use sequential matching for long trajectories
            if self.config['sequential_matching'] and len(points) > 50:
                return self._sequential_matching(point_objects)
            
            # Standard matching for shorter trajectories
            candidates_by_point = [self._find_candidates(p) for p in point_objects]
            
            if not all(candidates_by_point):
                return {'success': False, 'edges': [], 'confidence': 0.0}
            
            states = self._viterbi_matching(point_objects, candidates_by_point)
            path = self._backtrack(states)
            
            if not path:
                return {'success': False, 'edges': [], 'confidence': 0.0}
            
            confidence = states[-1][path[-1]].get('confidence', 0.0)
            
            return {
                'success': True,
                'edges': path,
                'confidence': confidence
            }
                
        except Exception as e:
            self.logger.error(f"Error in match_trajectory: {str(e)}")
            return {'success': False, 'edges': [], 'confidence': 0.0}




In [147]:
import pandas as pd
import numpy as np
from typing import List, Dict
from datetime import datetime
import os



In [148]:
class MapMatchingReportGenerator:
    """Enhanced report generator for map matching results"""
    
    def __init__(self, output_dir: str = 'map_matching_results'):
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)

    def generate_quality_metrics(self, matched_results: List[Dict], matcher) -> pd.DataFrame:
        """Generate quality metrics for matched trajectories"""
        metrics = []
        
        for idx, result in enumerate(matched_results):
            edges = result['match_result']['edges']
            confidence = result['match_result']['confidence']
            
            # Calculate path metrics
            path_length = sum(matcher.edges_gdf.iloc[edge_id].geometry.length 
                            for edge_id in edges)
            
            # Calculate continuity score
            connected_pairs = 0
            total_pairs = len(edges) - 1
            for i in range(total_pairs):
                edge1 = matcher.edges_gdf.iloc[edges[i]]
                edge2 = matcher.edges_gdf.iloc[edges[i+1]]
                nodes1 = set([edge1.geometry.coords[0], edge1.geometry.coords[-1]])
                nodes2 = set([edge2.geometry.coords[0], edge2.geometry.coords[-1]])
                if nodes1.intersection(nodes2):
                    connected_pairs += 1
            
            continuity_score = connected_pairs / max(total_pairs, 1)
            
            metrics.append({
                'trajectory_id': idx,
                'confidence': confidence,
                'path_length_meters': path_length,
                'num_segments': len(edges),
                'continuity_score': continuity_score,
                'avg_segment_length': path_length / len(edges)
            })
            
        df = pd.DataFrame(metrics)
        df.to_csv(os.path.join(self.output_dir, 'quality_metrics.csv'), index=False)
        return df

    def generate_trajectory_analysis(self, matched_results: List[Dict]) -> pd.DataFrame:
        """Generate trajectory-specific analysis"""
        analysis = []
        
        for idx, result in enumerate(matched_results):
            coords = np.array(result['original_coords'])
            
            # Calculate metrics
            distances = np.sqrt(np.sum((coords[1:] - coords[:-1])**2, axis=1))
            avg_distance = np.mean(distances)
            
            angles = []
            for i in range(len(coords) - 2):
                v1 = coords[i+1] - coords[i]
                v2 = coords[i+2] - coords[i+1]
                angle = np.arctan2(np.cross(v1, v2), np.dot(v1, v2))
                angles.append(abs(angle))
            
            analysis.append({
                'trajectory_id': idx,
                'num_points': len(coords),
                'matched_segments': len(result['match_result']['edges']),
                'confidence': result['match_result']['confidence'],
                'avg_point_distance': avg_distance,
                'avg_angle': np.mean(angles) if angles else 0,
                'max_angle': np.max(angles) if angles else 0
            })
        
        df = pd.DataFrame(analysis)
        df.to_csv(os.path.join(self.output_dir, 'trajectory_analysis.csv'), index=False)
        return df

    def generate_summary_report(self, quality_metrics: pd.DataFrame, trajectory_analysis: pd.DataFrame):
        """Generate comprehensive summary report"""
        report = "Map Matching Summary Report\n"
        report += "========================\n\n"
        
        # Overall statistics
        report += "Overall Statistics:\n"
        report += "-----------------\n"
        report += f"Total trajectories processed: {len(quality_metrics)}\n"
        report += f"Average confidence: {quality_metrics['confidence'].mean():.3f}\n"
        report += f"Average path length: {quality_metrics['path_length_meters'].mean():.1f} meters\n"
        report += f"Average continuity score: {quality_metrics['continuity_score'].mean():.3f}\n\n"
        
        # Quality metrics summary
        report += "Quality Metrics Summary:\n"
        report += "---------------------\n"
        report += quality_metrics.describe().to_string() + "\n\n"
        
        # Trajectory analysis summary
        report += "Trajectory Analysis Summary:\n"
        report += "-------------------------\n"
        report += trajectory_analysis.describe().to_string() + "\n"
        
        with open(os.path.join(self.output_dir, 'summary_report.txt'), 'w') as f:
            f.write(report)

    def generate_all_reports(self, matched_results: List[Dict], matcher):
        """Generate all reports in one go"""
        # Generate quality metrics and trajectory analysis
        quality_metrics = self.generate_quality_metrics(matched_results, matcher)
        trajectory_analysis = self.generate_trajectory_analysis(matched_results)
        
        # Generate summary report
        self.generate_summary_report(quality_metrics, trajectory_analysis)

def generate_all_reports(matched_results: List[Dict], matcher, output_dir: str = 'map_matching_results'):
    """Main function to generate all reports"""
    try:
        # Initialize report generator
        report_generator = MapMatchingReportGenerator(output_dir)
        
        # Generate all reports
        report_generator.generate_all_reports(matched_results, matcher)
        
        print(f"Successfully generated reports for {len(matched_results)} trajectories")
        print(f"Reports saved in {output_dir} folder")
        
    except Exception as e:
        print(f"Error generating reports: {str(e)}")
        raise

In [150]:
class EnhancedProbabilityReportGenerator:
    """Generate detailed probability analysis reports for map matching results"""
    
    def __init__(self, output_dir: str = 'map_matching_results'):
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        self.prob_dir = os.path.join(output_dir, 'probability_analysis')
        os.makedirs(self.prob_dir, exist_ok=True)
        
    def calculate_detailed_probabilities(self, matcher, trajectory_points: List[Point], 
                                      matched_edges: List[int], states: List[Dict]) -> Dict:
        """Calculate detailed probabilities for a matched trajectory"""
        
        detailed_probs = {
            'emission_probs': [],
            'transition_probs': [],
            'path_probs': [],
            'cumulative_probs': []
        }
        
        # Calculate emission probabilities for each point
        for point_idx, point in enumerate(trajectory_points):
            point_emissions = []
            edge_id = matched_edges[point_idx]
            edge_geom = matcher.edges_gdf.loc[edge_id].geometry
            
            # Get candidate edges and their probabilities
            candidates = matcher._find_candidates(point)
            for candidate in candidates:
                prob = matcher._calculate_emission_prob(point, candidate)
                point_emissions.append({
                    'edge_id': candidate['edge_id'],
                    'probability': prob,
                    'distance': candidate['distance'],
                    'is_matched': candidate['edge_id'] == edge_id
                })
            
            detailed_probs['emission_probs'].append({
                'point_idx': point_idx,
                'candidates': point_emissions
            })
        
        # Calculate transition probabilities between consecutive points
        for i in range(len(trajectory_points) - 1):
            curr_edge = matched_edges[i]
            next_edge = matched_edges[i + 1]
            
            trans_prob = matcher._calculate_transition_prob(
                curr_edge, next_edge,
                trajectory_points[i], trajectory_points[i + 1]
            )
            
            # Get additional transition details
            curr_nodes = matcher.edge_to_nodes[curr_edge]
            next_nodes = matcher.edge_to_nodes[next_edge]
            is_connected = bool(curr_nodes.intersection(next_nodes))
            
            detailed_probs['transition_probs'].append({
                'segment_idx': i,
                'from_edge': curr_edge,
                'to_edge': next_edge,
                'probability': trans_prob,
                'is_connected': is_connected
            })
        
        # Extract path probabilities from Viterbi states
        for t, state in enumerate(states):
            if matched_edges[t] in state:
                state_info = state[matched_edges[t]]
                detailed_probs['path_probs'].append({
                    'step': t,
                    'edge_id': matched_edges[t],
                    'log_prob': state_info['log_prob'],
                    'emission': state_info['emission'],
                    'transition': state_info.get('transition', 0.0)
                })
                
                # Calculate cumulative probability
                cum_prob = np.exp(state_info['log_prob'])
                detailed_probs['cumulative_probs'].append({
                    'step': t,
                    'cumulative_prob': cum_prob
                })
        
        return detailed_probs
    
    def generate_probability_report(self, match_result: Dict, matched_edges: List[int], 
                                  detailed_probs: Dict, trajectory_id: int):
        """Generate detailed probability report for a single trajectory"""
        
        report = f"Probability Analysis Report - Trajectory {trajectory_id}\n"
        report += "=" * 50 + "\n\n"
        
        # Overall match statistics
        report += "Overall Match Statistics:\n"
        report += "-----------------------\n"
        report += f"Final confidence score: {match_result['confidence']:.4f}\n"
        report += f"Number of matched segments: {len(matched_edges)}\n"
        report += f"Match success: {match_result['success']}\n\n"
        
        # Emission probability analysis
        report += "Emission Probability Analysis:\n"
        report += "---------------------------\n"
        for point_data in detailed_probs['emission_probs']:
            report += f"\nPoint {point_data['point_idx']}:\n"
            matched_candidate = None
            for candidate in point_data['candidates']:
                if candidate['is_matched']:
                    matched_candidate = candidate
                    report += f"* Selected edge {candidate['edge_id']}: "
                    report += f"prob = {candidate['probability']:.4f}, "
                    report += f"distance = {candidate['distance']:.2f}m\n"
                    break
            
            # Calculate statistics for alternatives
            alt_probs = [c['probability'] for c in point_data['candidates'] 
                        if not c['is_matched']]
            if alt_probs:
                report += f"  Alternative candidates: {len(alt_probs)}\n"
                report += f"  Max alternative prob: {max(alt_probs):.4f}\n"
                report += f"  Avg alternative prob: {np.mean(alt_probs):.4f}\n"
        
        # Transition probability analysis
        report += "\nTransition Probability Analysis:\n"
        report += "-----------------------------\n"
        for trans in detailed_probs['transition_probs']:
            report += f"\nSegment {trans['segment_idx']} → {trans['segment_idx']+1}:\n"
            report += f"* Edge {trans['from_edge']} → {trans['to_edge']}\n"
            report += f"* Probability: {trans['probability']:.4f}\n"
            report += f"* Connected: {'Yes' if trans['is_connected'] else 'No'}\n"
        
        # Path probability analysis
        report += "\nPath Probability Analysis:\n"
        report += "----------------------\n"
        for path_prob in detailed_probs['path_probs']:
            report += f"\nStep {path_prob['step']}:\n"
            report += f"* Edge: {path_prob['edge_id']}\n"
            report += f"* Log probability: {path_prob['log_prob']:.4f}\n"
            report += f"* Emission contribution: {path_prob['emission']:.4f}\n"
            report += f"* Transition contribution: {path_prob['transition']:.4f}\n"
        
        # Save the report
        filename = os.path.join(self.prob_dir, f'trajectory_{trajectory_id}_probability_analysis.txt')
        with open(filename, 'w') as f:
            f.write(report)
            
        return report
    
    def generate_summary_visualizations(self, all_detailed_probs: List[Dict], trajectory_ids: List[int]):
        """Generate summary visualizations of probability distributions"""
        
        # Set up the plotting environment
        plt.style.use('seaborn-v0_8-whitegrid')
        
        # 1. Emission Probability Distribution
        plt.figure(figsize=(12, 6))
        all_emissions = []
        for probs in all_detailed_probs:
            for point_data in probs['emission_probs']:
                all_emissions.extend([c['probability'] for c in point_data['candidates']])
        
        plt.hist(all_emissions, bins=50, alpha=0.7)
        plt.title('Distribution of Emission Probabilities')
        plt.xlabel('Probability')
        plt.ylabel('Frequency')
        plt.savefig(os.path.join(self.prob_dir, 'emission_probability_distribution.png'))
        plt.close()
        
        # 2. Transition Probability Distribution
        plt.figure(figsize=(12, 6))
        all_transitions = []
        for probs in all_detailed_probs:
            all_transitions.extend([t['probability'] for t in probs['transition_probs']])
        
        plt.hist(all_transitions, bins=50, alpha=0.7)
        plt.title('Distribution of Transition Probabilities')
        plt.xlabel('Probability')
        plt.ylabel('Frequency')
        plt.savefig(os.path.join(self.prob_dir, 'transition_probability_distribution.png'))
        plt.close()
        
        # 3. Cumulative Probability Evolution
        plt.figure(figsize=(12, 6))
        for idx, probs in enumerate(all_detailed_probs):
            cum_probs = [p['cumulative_prob'] for p in probs['cumulative_probs']]
            plt.plot(cum_probs, label=f'Trajectory {trajectory_ids[idx]}')
        
        plt.title('Evolution of Cumulative Probabilities')
        plt.xlabel('Step')
        plt.ylabel('Cumulative Probability')
        plt.legend()
        plt.savefig(os.path.join(self.prob_dir, 'cumulative_probability_evolution.png'))
        plt.close()

def generate_probability_reports(matched_results: List[Dict], matcher, output_dir: str = 'map_matching_results'):
    """Main function to generate all probability reports"""
    prob_reporter = EnhancedProbabilityReportGenerator(output_dir)
    all_detailed_probs = []
    trajectory_ids = []
    
    for idx, result in enumerate(matched_results):
        try:
            # Get the matched trajectory details
            trajectory_points = [Point(p) for p in result['original_coords']]
            matched_edges = result['match_result']['edges']
            
            # Calculate detailed probabilities
            detailed_probs = prob_reporter.calculate_detailed_probabilities(
                matcher, trajectory_points, matched_edges, result.get('states', [])
            )
            
            # Generate individual report
            prob_reporter.generate_probability_report(
                result['match_result'], matched_edges, detailed_probs, idx
            )
            
            all_detailed_probs.append(detailed_probs)
            trajectory_ids.append(idx)
            
        except Exception as e:
            logging.error(f"Error generating probability report for trajectory {idx}: {str(e)}")
    
    # Generate summary visualizations
    prob_reporter.generate_summary_visualizations(all_detailed_probs, trajectory_ids)

In [151]:
# Constants
BUFFER_SIZE = 0.01  # roughly 1km in degrees
UTM_CRS = 'EPSG:32629'  # UTM zone 29N for Porto

def process_trajectory_with_validation(polyline_str: str) -> Optional[List[tuple]]:
    """Enhanced trajectory processing with better validation"""
    try:
        if not isinstance(polyline_str, str):
            return None
            
        # Clean string and handle potential formatting issues
        cleaned_str = polyline_str.strip().replace(' ', '')
        if not (cleaned_str.startswith('[') and cleaned_str.endswith(']')):
            return None
            
        coords = ast.literal_eval(cleaned_str)
        if not coords or not isinstance(coords, list):
            return None
        
        valid_coords = []
        prev_coord = None
        
        for coord in coords:
            if not isinstance(coord, (list, tuple)) or len(coord) != 2:
                continue
                
            try:
                x, y = map(float, coord)
                
                # Basic coordinate validation
                if not (-180 <= x <= 180 and -90 <= y <= 90):
                    continue
                    
                # Check for duplicate points
                if prev_coord and (x, y) == prev_coord:
                    continue
                    
                # Check for unrealistic jumps
                if prev_coord:
                    dx = x - prev_coord[0]
                    dy = y - prev_coord[1]
                    dist = (dx * dx + dy * dy) ** 0.5
                    if dist > 0.1:  # About 11km at the equator
                        continue
                        
                valid_coords.append((x, y))
                prev_coord = (x, y)
                
            except (ValueError, TypeError):
                continue
        
        return valid_coords if len(valid_coords) >= 2 else None
        
    except Exception as e:
        logging.warning(f"Error processing trajectory: {str(e)}")
        return None

def validate_trajectory_bounds(coords: List[tuple], bounds_wgs84_buffered: tuple) -> bool:
    """Validate if trajectory is within buffered network bounds"""
    if not coords:
        return False
        
    coords_array = np.array(coords)
    min_x_coord = coords_array[:, 0].min()
    max_x_coord = coords_array[:, 0].max()
    min_y_coord = coords_array[:, 1].min()
    max_y_coord = coords_array[:, 1].max()
    
    return (
        bounds_wgs84_buffered[0] <= max_x_coord and 
        min_x_coord <= bounds_wgs84_buffered[2] and
        bounds_wgs84_buffered[1] <= max_y_coord and 
        min_y_coord <= bounds_wgs84_buffered[3]
    )

def convert_to_utm(coords: List[tuple], utm_crs: str) -> List[tuple]:
    """Convert coordinates from WGS84 to UTM"""
    point_gdf = gpd.GeoDataFrame(
        geometry=[Point(x, y) for x, y in coords],
        crs='EPSG:4326'
    ).to_crs(utm_crs)
    
    return [(p.x, p.y) for p in point_gdf.geometry]



def visualize_individual_matches(matcher, matched_results: List[Dict], output_dir: str):
    """Generate individual interactive visualizations for each matched route"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Convert edges to WGS84 for visualization
    edges_wgs84 = matcher.edges_gdf.to_crs('EPSG:4326')
    
    # Center coordinates for Porto
    center_lat, center_lon = 41.1579, -8.6291
    
    # Create color scheme
    colors = plt.cm.rainbow(np.linspace(0, 1, 15))  # Fixed for 15 trajectories
    colors = ['#%02x%02x%02x' % tuple(map(int, c[:3] * 255)) for c in colors]
    
    # Process each trajectory index
    processed_indices = set(result.get('trajectory_id', i) for i, result in enumerate(matched_results))
    
    # Generate individual maps for each trajectory
    for idx in range(15):  # Process all 15 trajectories
        # Create new base map for each trajectory
        individual_map = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=13,
            tiles='cartodbpositron'
        )
        
        # Add road network
        for _, edge in edges_wgs84.iterrows():
            if edge.geometry is not None:
                coords = [(y, x) for x, y in edge.geometry.coords]
                folium.PolyLine(
                    coords,
                    weight=1,
                    color='gray',
                    opacity=0.5
                ).add_to(individual_map)
        
        # Find the matching result for this trajectory index
        matched_result = None
        for result in matched_results:
            if result.get('trajectory_id', -1) == idx or (idx not in processed_indices and matched_results.index(result) == idx):
                matched_result = result
                break
        
        if matched_result:
            # Draw original trajectory
            coords = [(y, x) for x, y in matched_result['original_coords']]
            folium.PolyLine(
                coords,
                weight=2,
                color=colors[idx],
                opacity=0.5,
                popup='Original Trajectory'
            ).add_to(individual_map)
            
            # Draw matched edges
            for edge_id in matched_result['match_result']['edges']:
                edge = edges_wgs84.iloc[edge_id]
                if edge.geometry is not None:
                    coords = [(y, x) for x, y in edge.geometry.coords]
                    folium.PolyLine(
                        coords,
                        weight=3,
                        color=colors[idx],
                        opacity=0.8,
                        popup='Matched Route'
                    ).add_to(individual_map)
            
            # Add confidence score to legend if available
            confidence = matched_result['match_result'].get('confidence', 'N/A')
            status = "Successfully Matched"
        else:
            status = "Matching Failed"
            confidence = 'N/A'
        
        # Add legend with match status and confidence
        legend_html = f"""
            <div style="position: fixed; 
                        bottom: 50px; left: 50px; 
                        background-color: white;
                        border: 2px solid grey; 
                        z-index: 1000; 
                        padding: 10px">
                <p><strong>Trajectory {idx + 1}</strong></p>
                <p><span style="color: gray">―――</span> Road Network</p>
                <p><span style="color: {colors[idx]}">―――</span> Trajectory</p>
                <p>Status: {status}</p>
                <p>Confidence: {confidence}</p>
            </div>
        """
        individual_map.get_root().html.add_child(folium.Element(legend_html))
        
        # Save individual map
        individual_map.save(os.path.join(output_dir, f'trajectory_{idx + 1}.html'))



def main():
    """Main function for map matching pipeline"""
    
    # Set up logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    logger = logging.getLogger(__name__)

    try:
        # Load road network with a larger buffer
        logger.info("Loading road network...")
        G = ox.graph_from_place('Porto, Portugal', network_type='drive', buffer_dist=2000)
        nodes, edges = ox.graph_to_gdfs(G)
        
        # Get the original WGS84 bounds before conversion
        edges_wgs84 = edges.copy()
        bounds_wgs84 = edges_wgs84.total_bounds
        logger.info(f"Network bounds (WGS84): {bounds_wgs84}")
        
        # Add buffer to bounds
        bounds_wgs84_buffered = (
            bounds_wgs84[0] - BUFFER_SIZE,
            bounds_wgs84[1] - BUFFER_SIZE,
            bounds_wgs84[2] + BUFFER_SIZE,
            bounds_wgs84[3] + BUFFER_SIZE
        )
        logger.info(f"Buffered bounds (WGS84): {bounds_wgs84_buffered}")
        
        # Convert to UTM coordinates for accurate distance calculations
        edges = edges.to_crs(UTM_CRS)
        
        # Load trajectory data
        logger.info("Loading trajectory data...")
        df = pd.read_csv('kraggle_data/train/train.csv', nrows=15)
        logger.info(f"Loaded {len(df)} trajectories")
        
        # Initialize matcher
        logger.info("Initializing matcher...")
        config = {
            'max_candidates': 20,
            'max_distance': 300.0,
            'sigma_z': 75.0,
            'beta': 2.0,
            'min_prob_norm': 1e-4,
            'max_speed': 50.0,
            'min_speed': 0.1
        }
        
        matcher = EnhancedViterbiMatcher(G, edges, config)
        
        # Process trajectories
        logger.info("Processing trajectories...")
        matched_results = []
        failed_matches = []
        
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Matching trajectories"):
            try:
                # Process and validate trajectory
                coords = process_trajectory_with_validation(row['POLYLINE'])
                if not coords:
                    logger.warning(f"Failed to process trajectory {idx}: Invalid coordinates")
                    failed_matches.append({'id': idx, 'reason': 'Invalid coordinates'})
                    continue
                
                # Debug output for first few trajectories
                if idx < 3:
                    logger.debug(f"Trajectory {idx} bounds: "
                               f"X: [{min(x for x,_ in coords)}, {max(x for x,_ in coords)}], "
                               f"Y: [{min(y for _,y in coords)}, {max(y for _,y in coords)}]")
                
                # Validate trajectory bounds
                if not validate_trajectory_bounds(coords, bounds_wgs84_buffered):
                    logger.warning(f"Trajectory {idx} outside network bounds")
                    failed_matches.append({'id': idx, 'reason': 'Outside bounds'})
                    continue
                
                # Convert to UTM
                try:
                    utm_coords = convert_to_utm(coords, UTM_CRS)
                except Exception as e:
                    logger.error(f"Coordinate conversion error for trajectory {idx}: {str(e)}")
                    failed_matches.append({'id': idx, 'reason': 'Coordinate conversion error'})
                    continue
                
                # Match trajectory
                result = match_with_retry(matcher, utm_coords)
                if result['success']:
                        # Get the Viterbi states from the matcher
                        point_objects = [Point(p) for p in utm_coords]
                        candidates = [matcher._find_candidates(p) for p in point_objects]
                        states = matcher._viterbi_matching(point_objects, candidates)
                        
                        matched_results.append({
                            'match_result': result,
                            'original_coords': coords,
                            'trajectory_id': idx,
                            'states': states  # Add states to results
                        })
                        logger.info(f"Successfully matched trajectory {idx} "
                                f"(confidence: {result['confidence']:.3f})")
                else:
                        logger.warning(f"Failed to match trajectory {idx}")
                        failed_matches.append({'id': idx, 'reason': 'Matching failed'})
                    
                
            
                    
            except Exception as e:
                logger.error(f"Error processing trajectory {idx}: {str(e)}")
                failed_matches.append({'id': idx, 'reason': str(e)})
        
        # Generate reports
        logger.info("Generating reports...")
        # Add probability report generation
        generate_probability_reports(matched_results, matcher, output_dir='map_matching_results')
        # Generate failure analysis
        generate_failure_analysis(failed_matches, output_dir='map_matching_results')
        
        
        
        # print("Generating reports...")
        
        # print("Generating reports...")
        if matched_results:
            output_dir = 'map_matching_results'
            generate_all_reports(matched_results, matcher, output_dir)
            
            # Generate both overview and individual visualizations
            logger.info("Generating visualizations...")
            #visualize_matches(matcher, matched_results, output_dir)  # Overall visualization
            visualize_individual_matches(matcher, matched_results, output_dir)  # Individual visualizations
            
            print(f"Successfully matched {len(matched_results)} trajectories")
            print(f"Reports and visualizations saved in {output_dir} folder")
        else:
            print("No trajectories were successfully matched")
        
        
            
    except Exception as e:
        logger.error(f"Critical error in main process: {str(e)}", exc_info=True)
        raise


      

def process_trajectory_with_validation(polyline_str: str) -> Optional[List[tuple]]:
    """Enhanced trajectory processing with better validation"""
    try:
        if not isinstance(polyline_str, str):
            return None
            
        # Clean string and handle potential formatting issues
        cleaned_str = polyline_str.strip().replace(' ', '')
        if not (cleaned_str.startswith('[') and cleaned_str.endswith(']')):
            return None
            
        coords = ast.literal_eval(cleaned_str)
        if not coords or not isinstance(coords, list):
            return None
        
        valid_coords = []
        prev_coord = None
        
        for coord in coords:
            if not isinstance(coord, (list, tuple)) or len(coord) != 2:
                continue
                
            try:
                x, y = map(float, coord)
                
                # Basic coordinate validation
                if not (-180 <= x <= 180 and -90 <= y <= 90):
                    continue
                    
                # Check for duplicate points
                if prev_coord and (x, y) == prev_coord:
                    continue
                    
                # Check for unrealistic jumps
                if prev_coord:
                    dx = x - prev_coord[0]
                    dy = y - prev_coord[1]
                    dist = (dx * dx + dy * dy) ** 0.5
                    if dist > 0.1:  # About 11km at the equator
                        continue
                        
                valid_coords.append((x, y))
                prev_coord = (x, y)
                
            except (ValueError, TypeError):
                continue
        
        return valid_coords if len(valid_coords) >= 2 else None
        
    except Exception:
        return None

def match_with_retry(matcher: EnhancedViterbiMatcher, coords: List[tuple], max_attempts: int = 3) -> Dict:
    """Attempt matching with different parameters if initial attempt fails"""
    result = matcher.match_trajectory(coords)
    
    if result['success']:
        return result
    
    # Try with increased search radius
    for attempt in range(max_attempts - 1):
        increased_distance = matcher.config['max_distance'] * (1.5 ** (attempt + 1))
        increased_sigma = matcher.config['sigma_z'] * (1.2 ** (attempt + 1))
        
        temp_config = matcher.config.copy()
        temp_config.update({
            'max_distance': increased_distance,
            'sigma_z': increased_sigma
        })
        
        temp_matcher = EnhancedViterbiMatcher(
            matcher.graph, 
            matcher.edges_gdf,
            config=temp_config
        )
        
        result = temp_matcher.match_trajectory(coords)
        if result['success']:
            return result
    
    return {'success': False, 'edges': [], 'confidence': 0.0}

def generate_failure_analysis(failed_matches: List[Dict], output_dir: str):
    """Generate detailed analysis of matching failures"""
    os.makedirs(output_dir, exist_ok=True)
    
    report = "Matching Failure Analysis\n"
    report += "=======================\n\n"
    
    # Group failures by reason
    failure_reasons = {}
    for failure in failed_matches:
        reason = failure['reason']
        if reason not in failure_reasons:
            failure_reasons[reason] = []
        failure_reasons[reason].append(failure['id'])
    
    # Generate summary
    report += "Summary of Failures:\n"
    report += "-----------------\n"
    total_failures = len(failed_matches)
    
    for reason, trajectories in failure_reasons.items():
        count = len(trajectories)
        percentage = (count / total_failures) * 100
        report += f"\n{reason}:\n"
        report += f"Count: {count} ({percentage:.1f}%)\n"
        report += f"Affected trajectories: {sorted(trajectories)}\n"
    
    # Add overall statistics
    report += "\nOverall Statistics:\n"
    report += "-----------------\n"
    report += f"Total failures: {total_failures}\n"
    report += f"Unique failure reasons: {len(failure_reasons)}\n"
    
    with open(os.path.join(output_dir, 'failure_analysis.txt'), 'w') as f:
        f.write(report)
    
    # Create a visualization of failure distribution
    try:
        plt.figure(figsize=(10, 6))
        reasons = list(failure_reasons.keys())
        counts = [len(trajectories) for trajectories in failure_reasons.values()]
        
        plt.bar(reasons, counts)
        plt.title('Distribution of Matching Failures')
        plt.xlabel('Failure Reason')
        plt.ylabel('Number of Failures')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        
        plt.savefig(os.path.join(output_dir, 'failure_distribution.png'))
        plt.close()
    except Exception as e:
        print(f"Warning: Could not generate failure distribution plot: {str(e)}")



if __name__ == "__main__":
    main()

INFO:__main__:Loading road network...
/var/folders/_6/2sxd3q_d65b9mp5y1l5_0g7r0000gn/T/ipykernel_56150/2454098061.py:216: FutureWarning: The buffer_dist argument has been deprecated and will be removed in the v2.0.0 release. Buffer your query area directly, if desired. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  G = ox.graph_from_place('Porto, Portugal', network_type='drive', buffer_dist=2000)
/Users/macbookpro/miniconda3/envs/urbancom2/lib/python3.11/site-packages/osmnx/graph.py:387: FutureWarning: The buffer_dist argument has been deprecated and will be removed in the v2.0.0 release. Buffer your results directly, if desired. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_place = geocoder.geocode_to_gdf(
INFO:__main__:Network bounds (WGS84): [-8.6968313 41.1205698 -8.5288196 41.2038855]
INFO:__main__:Buffered bounds (WGS84): (-8.7068313, 41.1105698, -8.5188196, 41.213885499999996)
INFO:__main__:Loading trajecto

Successfully generated reports for 15 trajectories
Reports saved in map_matching_results folder
Successfully matched 15 trajectories
Reports and visualizations saved in map_matching_results folder


In [127]:
def analyze_failed_trajectories():
    """Analyze failed trajectories from the training data"""
    try:
        # Load the original data
        df = pd.read_csv('kraggle_data/train/train.csv', nrows=15)
        
        # Initialize report
        report = "Detailed Analysis of Failed Trajectories\n"
        report += "===================================\n\n"
        
        # Known failed indices from the logs
        failed_indices = [2, 8, 14]
        
        for idx in failed_indices:
            try:
                trajectory = df.iloc[idx]
                coords = process_trajectory_with_validation(trajectory['POLYLINE'])
                
                report += f"Trajectory {idx}:\n"
                report += "-" * 20 + "\n"
                
                if coords is None:
                    report += "Failed validation - Invalid coordinate format\n\n"
                    continue
                    
                coords_array = np.array(coords)
                
                # Analyze coordinate bounds
                x_min, y_min = coords_array.min(axis=0)
                x_max, y_max = coords_array.max(axis=0)
                
                # Network bounds from the logs
                network_bounds = {
                    'x_min': -8.7068313,
                    'x_max': -8.5188196,
                    'y_min': 41.1105698,
                    'y_max': 41.2138855
                }
                
                report += f"Coordinate bounds:\n"
                report += f"  X: [{x_min:.6f}, {x_max:.6f}]\n"
                report += f"  Y: [{y_min:.6f}, {y_max:.6f}]\n"
                
                # Check if trajectory is within network bounds
                within_bounds = (
                    network_bounds['x_min'] <= x_min <= network_bounds['x_max'] and
                    network_bounds['x_min'] <= x_max <= network_bounds['x_max'] and
                    network_bounds['y_min'] <= y_min <= network_bounds['y_max'] and
                    network_bounds['y_min'] <= y_max <= network_bounds['y_max']
                )
                
                if not within_bounds:
                    report += "  WARNING: Trajectory outside network bounds\n"
                    # Calculate how far outside bounds
                    x_outside = max(0, network_bounds['x_min'] - x_min,
                                  x_max - network_bounds['x_max'])
                    y_outside = max(0, network_bounds['y_min'] - y_min,
                                  y_max - network_bounds['y_max'])
                    if x_outside > 0:
                        report += f"    X out of bounds by: {x_outside:.6f} degrees\n"
                    if y_outside > 0:
                        report += f"    Y out of bounds by: {y_outside:.6f} degrees\n"
                
                # Analyze trajectory characteristics
                num_points = len(coords)
                distances = np.sqrt(np.sum((coords_array[1:] - coords_array[:-1])**2, axis=1))
                avg_distance = np.mean(distances)
                max_distance = np.max(distances)
                
                report += f"\nTrajectory characteristics:\n"
                report += f"  Number of points: {num_points}\n"
                report += f"  Average distance between points: {avg_distance:.6f} degrees\n"
                report += f"  Maximum distance between points: {max_distance:.6f} degrees\n"
                
                # Calculate angles between consecutive segments
                if len(coords) >= 3:
                    angles = []
                    for i in range(len(coords) - 2):
                        v1 = coords_array[i+1] - coords_array[i]
                        v2 = coords_array[i+2] - coords_array[i+1]
                        angle = np.arctan2(np.cross(v1, v2), np.dot(v1, v2))
                        angles.append(abs(angle))
                    
                    max_angle = max(angles)
                    avg_angle = np.mean(angles)
                    report += f"  Average angle between segments: {np.degrees(avg_angle):.2f} degrees\n"
                    report += f"  Maximum angle between segments: {np.degrees(max_angle):.2f} degrees\n"
                
                # Check for potential issues
                if max_distance > 0.01:  # ~ 1km
                    report += "  WARNING: Contains large jumps between points\n"
                    
                if num_points < 3:
                    report += "  WARNING: Very few points for matching\n"
                    
                if any(d < 1e-5 for d in distances):  # Very close points
                    report += "  WARNING: Contains nearly duplicate points\n"
                
                report += "\n"
                
            except Exception as e:
                report += f"Error analyzing trajectory {idx}: {str(e)}\n\n"
        
        # Save report
        output_dir = 'map_matching_results'
        os.makedirs(output_dir, exist_ok=True)
        with open(os.path.join(output_dir, 'failed_trajectories_analysis.txt'), 'w') as f:
            f.write(report)
            
        return report
        
    except Exception as e:
        return f"Error loading or processing data: {str(e)}"

# Generate the analysis
failure_analysis = analyze_failed_trajectories()
print(failure_analysis)

Detailed Analysis of Failed Trajectories

Trajectory 2:
--------------------
Coordinate bounds:
  X: [-8.650395, -8.596530]
  Y: [41.140278, 41.154507]

Trajectory characteristics:
  Number of points: 65
  Average distance between points: 0.003155 degrees
  Maximum distance between points: 0.055281 degrees
  Average angle between segments: 28.49 degrees
  Maximum angle between segments: 172.08 degrees

Trajectory 8:
--------------------
Coordinate bounds:
  X: [-8.615844, -8.589402]
  Y: [41.140341, 41.163453]

Trajectory characteristics:
  Number of points: 38
  Average distance between points: 0.001234 degrees
  Maximum distance between points: 0.002874 degrees
  Average angle between segments: 29.28 degrees
  Maximum angle between segments: 134.96 degrees

Trajectory 14:
--------------------
Coordinate bounds:
  X: [-8.616024, -8.606772]
  Y: [41.140287, 41.158260]

Trajectory characteristics:
  Number of points: 28
  Average distance between points: 0.001065 degrees
  Maximum dista